<a href="https://colab.research.google.com/github/hyunwoo94/DataScience/blob/main/%EB%84%A4%EC%9D%B4%EB%B2%84_%EC%98%81%ED%99%94_TOP100%ED%81%AC%EB%A1%A4%EB%A7%81%26%EC%98%81%ED%99%94_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#링크 수집을 위한 라이브러리 탐색
!pip install selenium

     |████████████████████████████████| 911kB 24.9MB/s 


라이브러리 임포트

In [3]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver as wd 
from selenium.webdriver.common.keys import Keys
import time
import re
import pandas as pd

In [4]:
# install chromium, its driver, and selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)


Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15

분석할 데이터 모으기(크롤링)

네이버 영화 TOP100 페이지 별 링크 수집

In [12]:
#https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_R&page= 실시간
#https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_R&page= 일간
#https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_R&page= 주간
#https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_R&page= 월간
#https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_R&page=

def getPageLinksWantRange(startPageNo, lastPageNo) :
  links = []        #100개의 영화 링크를 수집할 리스트 변수 선언
  return_links =[]
  
  for pageNo in range(startPageNo -1, lastPageNo) :
    # 실시간 TOP 100 영화 링크 수집
    url = "https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_R&page=" + str(pageNo + 1)
    req =requests.get(url)      # 페이지 접속
    soup =BeautifulSoup(req.text,'lxml')
    movielinks = soup.select('div.lst_thum_wrap ul li a[href]') # <div class="lst_thum_wrap"> -> <ul> -> <li> -> <a href ="">


    for movielink in movielinks:
      link = str(movielink.get('href'))
      links.append("https://series.naver.com" +link)
  return links

네이버 영화 제목, 평점, 장르, 줄거리 크롤링

In [13]:
# 네이버 영화 정보 크롤링 함수
def getMovieDataFromNaverSeries(links):
  title_infos =[]     # 제목
  content_infos = []  # 줄거리
  genre_infos = []    # 장르
  score_infos = []    # 평정
  date_infos = []     # 개봉일
  
  url2 = "https://www.naver.com"

  driver = webdriver.Chrome('chromedriver', options=options)
  driver.get(url2)
  time.sleep(3.0) # 3초 sleep

  driver.find_element_by_css_selector('body').send_keys(Keys.CONTROL + "t")

  for link in links:
    print(link + ' 수집 중........')
    driver.switch_to.window(driver.window_handles[-1])  # 새창 활성화
    time.sleep(0.1)
    driver.get(link)
    time.sleep(0.1)
    driver.switch_to.window(driver.window_handles[0])
    time.sleep(0.3)

    html_source = driver.page_source   # req.text 같은 동작

    html_soup = BeautifulSoup(html_source, 'lxml')

    flag = html_soup.text[0:10]
    # 청소년관람불가 영화 크롤링 할 경우 -> 인증 -> 수집 제외
    newflag = "".join(flag)
    newflag = newflag.replace('\n', '')

    if newflag == '네이버':
      time.sleep(1.0)

      # 평점 수집
      score = driver.find_element_by_css_selector('div.score_area > em ')
      score = float(score.text)
      score = int(score)
      score_infos.append(score)
      print(score)

      # 장르 수집
      genre = driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(4)').get_attribute('textContent')
      genre = genre.replace('장르','')
      genre = genre.replace('\n','')
      genre = genre.replace('\t','')
      genre_infos.append(genre)
      print(genre)

      #제목, 줄거리

      try:
        movieInfoUrl = driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(7) > a').get_attribute('href')
        # 장르 수집 -> 장르 정보가 없어서, 6번째 except ->  공백으로 수집
      except:
        movieInfoUrl = driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(6) > a').get_attribute('href')
      
      # 영화 상세보기 페이지로 이동
      movie_req = requests.get(movieInfoUrl)

      # 제목 수집
      movie_soup = BeautifulSoup(movie_req.text,'lxml')
      title = movie_soup.head.find("meta", {"property":"og:title"}).get('content')
      title_infos.append(title)
      print(title)

      # 줄거리 수집
      contents_texts =movie_soup.select('div.story_area > p.con_tx')

      if len(contents_texts) == 0:
        content_infos.append("줄거리 오류")

      else:
        for contents in contents_texts:
          #줄거리 데이터 클랜징 작업
          temp = contents.text
          temp = temp.replace('\r','')   #줄바꿈 제거
          temp = temp.replace('\xa0', '')  # 공백제거
          content_infos.append(temp)
    elif newflag == '' : 
      print('청불 영화로 데이터 수집하지 않습니다.')
  print('수집 완료 합니다.......')
  print(len(score_infos), len(genre_infos), len(content_infos))
  driver.close()

  movie_dic = {"제목":title_infos, "평점": score_infos, "장르":genre_infos, "줄거리":content_infos}
  
  # 딕셔너리 -> DataFrame
  movie_df = pd.DataFrame(movie_dic)

  # 수집된 정보 중에 중복 데이터 삭제
  movie_df2 = movie_df.drop_duplicates("줄거리", keep = 'first') # 줄거리 비교해서, 중복된 영화를 삭제 (첫번쨰 수집 영화만 남김)

  return movie_df2


csv 파일로 저장

In [ ]:
def dftoCsv(movie_df,num):
  try:
    movie_df.to_csv(('movie_my_data_'+ str(num)+'.csv'),
                    sep=',', na_rep='NaN', encoding='euc-kr') #utf-8 혹시 그래도 한글 깨지면  encoding='utf-8-sig'

    print("Error")

크롤링 실행

In [ ]:
#1) 크롤링 할 링크 수집
links = getPageLinksWantRange(1,5) 
 
#2) 영화 상세 정보 수집
movie_df2 = getMovieDataFromNaverSeries(links)

#3)  csv 파일 저장
dftoCsv(movie_df2, len(movie_df2))

데이터 준비하기

In [ ]:
df = pd.read_csv('movie_my_data_75.csv', encoding ='euc-kr')
df.info()

In [ ]:
import plotly.express as px
fig = px.scatter(df, x='제목', y='평점', color='장르')
fig.show()

In [ ]:
df_score = df[df.평점 == 9]
df_score.tail()

9점의 영화 중 가장 큰 비율을 차지하고 있는 장르는?

In [ ]:
fig = px.pie(df_score,values='평점', names='장르')
fig.show()

일간 TOP 100 시간화

In [ ]:
df2 = pd.read_csv('movie_my_data_75.csv', encoding ='euc-kr')
df2.head()

In [ ]:
fig = px.bar(df2, x="제목", y='평점',color='장르')
fig.show()

In [ ]:
fig = px.pie(df2, values='평점', names='장르')
fig.show()

In [ ]:
# 월간 분석
df2 = pd.read_csv('movie_my_data_64_month.csv', encoding='euc-kr')
df2.head()